In [ ]:
import math
from itertools import product

In [ ]:
def deg2num(lat_deg, lon_deg, zoom):
    '''
        Convert latitude, longitude to (x,y) tile coordinate at given zoom.
    '''
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

In [ ]:
def num2deg(xtile, ytile, zoom):
    '''
        Convert (x,y) to latitude, longitude tile coordinate at given zoom.
    '''
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

In [ ]:
def tiles_url(zoom, lat1, lon1, lat2, lon2):
    ''' 
        Convert geographic bounds into a list of tile_url coordinates at given zoom.
    '''
    # convert to geographic bounding box
    minlat, minlon = min(lat1, lat2), min(lon1, lon2)
    maxlat, maxlon = max(lat1, lat2), max(lon1, lon2)

    # convert to tile-space bounding box
    xmin, ymin = deg2num(maxlat, minlon, zoom)
    xmax, ymax = deg2num(minlat, maxlon, zoom)

    # generate a list of tiles_url
    xs, ys = range(xmin, xmax+1), range(ymin, ymax+1)
    tiles_url = [(f"s3://elevation-tiles-prod//terrarium/{zoom}/{x}/{y}.png") for (y, x) in product(ys, xs)]

    return tiles_url

In [ ]:
data = tiles_url(11, 30, -85, 24, -79)

In [ ]:
df = spark.read.format("image").load(data)
df.show()